## import pkg

In [1]:
import sktime

### for preprocessing, we have different methods for different column (horizontal)

In [2]:
from sklearn.compose import ColumnTransformer

### for model training, we have step-by-step methods (vertical)

In [ ]:
from sklearn.pipeline import make_pipeline

### for Nan Numerical data in dp, i use KNNImputer

In [3]:
from sklearn.impute import KNNImputer

### for categorical data, i use OneHotEncoder

In [4]:
from sklearn.preprocessing import OneHotEncoder

### for numerical data, i use stdscaler

In [5]:
from sklearn.preprocessing import StandardScaler

## load dataset

In [9]:
path = "D:/202209/tbrain/訓練資料集_first/"
import os
os.chdir(path)
# import glob
# print(glob.glob(os.path.join(path, "*.csv")))
## 備註
## 這邊利用 os.path.join 來連接指定字串形成路徑，因為不同系統下的分隔符號可能不同

In [7]:
import pandas as pd

In [10]:
ccba = pd.read_csv("public_train_x_ccba_full_hashed.csv")
cdtx = pd.read_csv("public_train_x_cdtx0001_full_hashed.csv")
custinfo = pd.read_csv("public_train_x_custinfo_full_hashed.csv")
dp = pd.read_csv("public_train_x_dp_full_hashed.csv")
remit = pd.read_csv("public_train_x_remit1_full_hashed.csv")
alert_public = pd.read_csv("public_x_alert_date.csv")
alert_train = pd.read_csv("train_x_alert_date.csv")
y = pd.read_csv("train_y_answer.csv")

## Prepare training/testing target

In [11]:
train = y.join(alert_train.set_index('alert_key'), on='alert_key')
train = train.join(custinfo.set_index('alert_key'), on='alert_key')
train.head()

,alert_key,sar_flag,date,cust_id,risk_rank,occupation_code,total_asset,AGE
0,171189,0,0,972ee157e63316e8a50dd489bc93730a3ee8a8959d5c6b...,1,17.0,375576.0,4
1,171202,0,0,a10ab33f90926fb18d7bb5e78034d7f04a1fbed95b7951...,3,12.0,2717416.0,2
2,171599,0,0,3433ecc068ed1c9e2f5037cab5d42d7b901e9bd624c1fb...,1,12.0,326517.0,4
3,171737,0,0,a0861608615a9365d90f4ba0a813c0ea0471987f925c8b...,3,14.0,1014759.0,4
4,171142,0,0,a39fea9aec90969fe66a2b2b4d1b86368a2d38e8b8d4bf...,3,12.0,241719.0,3


In [12]:
test = y.join(alert_public.set_index('alert_key'), on='alert_key')
test = test.join(custinfo.set_index('alert_key'), on='alert_key')
test.head()

,alert_key,sar_flag,date,cust_id,risk_rank,occupation_code,total_asset,AGE
0,171189,0,NaN,972ee157e63316e8a50dd489bc93730a3ee8a8959d5c6b...,1,17.0,375576.0,4
1,171202,0,NaN,a10ab33f90926fb18d7bb5e78034d7f04a1fbed95b7951...,3,12.0,2717416.0,2
2,171599,0,NaN,3433ecc068ed1c9e2f5037cab5d42d7b901e9bd624c1fb...,1,12.0,326517.0,4
3,171737,0,NaN,a0861608615a9365d90f4ba0a813c0ea0471987f925c8b...,3,14.0,1014759.0,4
4,171142,0,NaN,a39fea9aec90969fe66a2b2b4d1b86368a2d38e8b8d4bf...,3,12.0,241719.0,3


In [ ]:
joint_

## prepare sub-dataset

## prepare dp

In [13]:
# calculate into NTD
dp['NTD'] = dp['tx_amt']*dp['exchg_rate'].round(2)

# is it originally used NTD?
dp['isNTD'] = dp.loc[dp["exchg_rate"] == 1.0, "isNTD"] = 1
dp['isNTD'] = dp.loc[dp["exchg_rate"] != 1.0, "isNTD"] = 0

dp[dp['exchg_rate']!=1.0].head()

,cust_id,debit_credit,tx_date,tx_time,tx_type,tx_amt,exchg_rate,info_asset_code,fiscTxId,txbranch,cross_bank,ATM,NTD,isNTD
199,01b1735c3dbf0a35c1c989bb45434ce4326b365b73db06...,CR,230,14,2,3715.0,27.83,20,NaN,NaN,0,1,103388.45,0
200,01b1735c3dbf0a35c1c989bb45434ce4326b365b73db06...,CR,231,9,2,5029.0,27.80,20,NaN,NaN,0,1,139806.20,0
201,01b1735c3dbf0a35c1c989bb45434ce4326b365b73db06...,DB,231,14,2,3715.0,27.84,20,NaN,NaN,0,1,103425.60,0
202,01b1735c3dbf0a35c1c989bb45434ce4326b365b73db06...,DB,231,14,2,8428.0,27.84,20,NaN,NaN,0,1,234635.52,0
205,01b1735c3dbf0a35c1c989bb45434ce4326b365b73db06...,DB,291,11,2,2064.0,27.63,20,NaN,NaN,0,1,57028.32,0


In [22]:
dp[dp['tx_amt'].isnull()][:10]

,cust_id,debit_credit,tx_date,tx_time,tx_type,tx_amt,exchg_rate,info_asset_code,fiscTxId,txbranch,cross_bank,ATM,NTD,isNTD
183640,235fc2d993edddded8510d46ebd3928382423b7e0d6a72...,DB,249,15,2,NaN,27.790,20,NaN,NaN,0,1,NaN,0
183641,235fc2d993edddded8510d46ebd3928382423b7e0d6a72...,CR,256,14,2,NaN,27.770,20,NaN,NaN,0,1,NaN,0
183642,235fc2d993edddded8510d46ebd3928382423b7e0d6a72...,CR,249,15,2,NaN,27.750,20,NaN,NaN,0,1,NaN,0
183643,235fc2d993edddded8510d46ebd3928382423b7e0d6a72...,DB,243,14,2,NaN,27.830,20,NaN,NaN,0,1,NaN,0
183644,235fc2d993edddded8510d46ebd3928382423b7e0d6a72...,DB,252,15,2,NaN,27.750,20,NaN,NaN,0,1,NaN,0
183645,235fc2d993edddded8510d46ebd3928382423b7e0d6a72...,CR,251,15,2,NaN,27.750,20,NaN,NaN,0,1,NaN,0
183646,235fc2d993edddded8510d46ebd3928382423b7e0d6a72...,CR,245,14,2,NaN,27.760,20,NaN,NaN,0,1,NaN,0
183647,235fc2d993edddded8510d46ebd3928382423b7e0d6a72...,CR,243,14,2,NaN,27.780,20,NaN,NaN,0,1,NaN,0
183648,235fc2d993edddded8510d46ebd3928382423b7e0d6a72...,CR,249,9,2,NaN,27.745,20,NaN,NaN,0,1,NaN,0
183649,235fc2d993edddded8510d46ebd3928382423b7e0d6a72...,CR,252,15,2,NaN,27.720,20,NaN,NaN,0,1,NaN,0


In [25]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=2, weights="uniform")

dp["tx_amt"] = imputer.fit_transform(dp[["tx_amt"]]).ravel()
